In [2]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
from scipy.optimize import minimize
import scipy.stats as stats
import time
import math
from statsmodels.iolib.summary2 import summary_col
import statsmodels.api as sm
import matplotlib.pyplot as plt

D:\Programs\Anacondaa\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


ln(wi,t) = α + β1Educi,t + β2Agei,t + β3Blacki,t + β4Hispanici,t + β5OtherRacei,t + εi,t

where: • wi,t = wage of individual i in survey year t 
       • Educi,t = education in years 
       • Agei,t = age in years 
       • Blacki,t, Hispanici,t, OtherRacei,t = dummy variables for race = Black, Hispanic, Not∈{White, Black, Hispanic}. 

# DATA PREPERATION

In [3]:
#read the stata file that has data
df1=pd.read_stata('PS3_data.dta')

df1.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
count,123786.000000,123786.000000,123786.000000,123786.000000,123786.000000,9.023300e+04,48496.000000,123786.000000,90603.000000,123656.000000,...,1.237860e+05,123786.000000,80758.0,123786.000000,80758.000000,123786.000000,80758.000000,122809.000000,80091.000000,123786.000000
mean,1494.639475,1984.831273,3271.379429,1679.269897,633.026917,4.211505e+04,22026.289062,0.843771,1.098220,1.129731,...,3.012258e+04,1.233072,2.0,45.545547,41.390785,39.620201,55.346169,12.666091,12.720081,0.557690
std,838.901790,9.836212,2277.056058,1061.704712,878.422791,4.670424e+04,21336.107422,1.182829,0.356161,0.394627,...,4.588795e+04,0.422940,0.0,17.623671,14.786721,69.003265,77.864296,2.917721,2.422607,0.265198
min,1.000000,1967.000000,1.000000,0.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,-1.324040e+05,1.000000,2.0,16.000000,13.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,772.000000,1977.000000,1444.000000,832.000000,0.000000,1.979858e+04,8016.246948,0.000000,1.000000,1.000000,...,7.700000e+03,1.000000,2.0,31.000000,29.000000,1.000000,2.000000,12.000000,12.000000,0.362158
50%,1517.000000,1985.000000,2984.000000,1976.000000,0.000000,3.460022e+04,18122.412109,0.000000,1.000000,1.000000,...,1.900000e+04,1.000000,2.0,42.000000,39.000000,3.000000,3.000000,12.000000,12.000000,0.599887
75%,2224.000000,1993.000000,4763.000000,2350.000000,1454.000000,5.267309e+04,30256.060547,2.000000,1.000000,1.000000,...,3.910775e+04,1.000000,2.0,58.000000,51.000000,22.000000,170.000000,15.000000,14.000000,0.786908
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,856942.062500,11.000000,8.000000,8.000000,...,3.660000e+06,2.000000,2.0,102.000000,95.000000,227.000000,231.000000,17.000000,17.000000,0.928007


In [4]:
df1.shape

(123786, 52)

In [5]:
#dropping missing values
df1 = df1[df1.hlabinc.isnull() !=True] #drop if missing hlabinc
df1 = df1[df1.hannhrs.isnull() !=True] #drop if missing hannhrs
df1 = df1[df1.age.isnull() !=True] #drop if missing age
df1 = df1[df1.hyrsed.isnull() !=True] #drop if missing educ

In [6]:
#creating logarithm of wage of individual i in survey year t
df1['hours_new'] = df1['hannhrs'].where(df1['hannhrs'] > 0)  #we are eliminating the possibility of getting infinity (because if we divide a number by zero we get infinity)
df1['wage_hours'] = df1['hlabinc']/df1['hours_new']
df1['ln_wage_hours'] = np.log(df1['wage_hours'])   # log of wages


#add constant to the data set
df1['const'] = 1  

#select only male heads of household who are between 25 and 60 years of age and earn wages > $7/hr  --> I defined new data called df2
df2 = df1[(df1['hsex']==1) & (df1['age'] <60) & (df1['age'] > 25) & (df1['wage_hours'] > 7)] 

#creating race dummies 
race_dummy = pd.get_dummies(df2['hrace'])
data_final = pd.concat([df2, race_dummy], axis=1)
data_final.rename (columns = {1.0:'White',2.0:'Black',3.0:'Others'},inplace=True)   #data_final is the final data

In [7]:
#CREATING DATA FOR EACH YEARS THAT IS ASKED
df1971 = data_final[data_final['year'] == 1971]
df1980 = data_final[data_final['year'] == 1980]
df1990 = data_final[data_final['year'] == 1990]
df2000 = data_final[data_final['year'] == 2000]

# Estimate the following model via a Maximum Likelihood Estimator separately for t = 1971, 1980, 1990, 2000

# year 1971

In [8]:
#MAXIMUM LIKELIHOOD ESTIMATIONS 

def regressLL(params):
    #the initial parameter guesses
    b0 = params[0]
    b1 = params[1]
    b2 = params[2]
    b3 = params[3]
    b4 = params[4]
    sd = params[5]

    # Calculate the predicted values from the initial parameter guesses
    yPred = b0 + b1 * df1971['hyrsed'] + b2 * df1971['age'] + b3 * df1971['Black'] + b4 * df1971['Others'] 

    # Calculate the negative log-likelihood as the negative sum of the log of a normal
    # PDF where the observed values are normally distributed around the mean (yPred)
    # with a standard deviation of sd
    logLik = -np.sum( stats.norm.logpdf(df1971['ln_wage_hours'], loc=yPred, scale=sd) )

    # Tell the function to return the NLL (this is what will be minimized)
    return(logLik)

# Make a list of initial parameter guesses (for the betas and  sd)    
initParams = [1, -6, 0, 0.5, -1, 1]

results = opt.minimize(regressLL, initParams, method='Nelder-Mead')



print(results)

 final_simplex: (array([[ 1.66657073,  0.06638463,  0.01194481, -0.18155526,  0.02444816,
         0.41192136],
       [ 1.66661452,  0.06638027,  0.0119449 , -0.18155701,  0.02451123,
         0.41192607],
       [ 1.66654164,  0.06638624,  0.01194503, -0.18153915,  0.02449545,
         0.41192408],
       [ 1.66653824,  0.06638619,  0.01194492, -0.1815699 ,  0.02444417,
         0.41192234],
       [ 1.66653391,  0.06638786,  0.01194476, -0.18155847,  0.02441683,
         0.41192744],
       [ 1.66654524,  0.0663861 ,  0.01194519, -0.18160341,  0.02449237,
         0.41192023],
       [ 1.66653617,  0.06638567,  0.01194522, -0.18154037,  0.02453569,
         0.41192278]]), array([ 679.41826758,  679.41827047,  679.41827094,  679.41827099,
        679.41828388,  679.41828589,  679.41828603]))
           fun: 679.41826758376942
       message: 'Optimization terminated successfully.'
          nfev: 1134
           nit: 711
        status: 0
       success: True
             x: array([ 

In [9]:
#OLS ESTIMATIONS

reg1 = sm.OLS(endog=df1971['ln_wage_hours'], exog=df1971[['const', 'hyrsed', 'age', 'Black', 'Others']], missing='drop')
type(reg1)
results = reg1.fit()
type(results)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          ln_wage_hours   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     99.30
Date:                Tue, 26 Sep 2017   Prob (F-statistic):           1.40e-73
Time:                        09:43:43   Log-Likelihood:                -678.67
No. Observations:                1274   AIC:                             1367.
Df Residuals:                    1269   BIC:                             1393.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.6049      0.077     20.866      0.0

# year 1980

In [10]:
#MAXIMUM LIKELIHOOD ESTIMATIONS 
def regressLL(params):
    #the initial parameter guesses
    b0 = params[0]
    b1 = params[1]
    b2 = params[2]
    b3 = params[3]
    b4 = params[4]
    sd = params[5]

    # Calculate the predicted values from the initial parameter guesses
    yPred = b0 + b1 * df1980['hyrsed'] + b2 * df1980['age'] + b3 * df1980['Black'] + b4 * df1980['Others'] 

    # Calculate the negative log-likelihood as the negative sum of the log of a normal
    # PDF where the observed values are normally distributed around the mean (yPred)
    # with a standard deviation of sd
    logLik = -np.sum( stats.norm.logpdf(df1980['ln_wage_hours'], loc=yPred, scale=sd) )

    # Tell the function to return the NLL (this is what will be minimized)
    return(logLik)

# Make a list of initial parameter guesses (for the betas and  sd)    
initParams = [1, -6, 0, 0.5, -1, 1]


results = opt.minimize(regressLL, initParams, method='Nelder-Mead')

print(results)


 final_simplex: (array([[ 1.63303999,  0.06762514,  0.01233048, -0.09267798,  0.03496745,
         0.45022863],
       [ 1.63301665,  0.06762715,  0.01233041, -0.09266758,  0.03495236,
         0.45023395],
       [ 1.63303228,  0.06762694,  0.01233017, -0.09268213,  0.03490972,
         0.45023328],
       [ 1.63304735,  0.06762568,  0.01233037, -0.09270176,  0.03494169,
         0.45023135],
       [ 1.63302714,  0.06762696,  0.01233028, -0.092657  ,  0.03493851,
         0.45023046],
       [ 1.63302871,  0.06762681,  0.0123303 , -0.09267913,  0.03493893,
         0.45023892],
       [ 1.6330725 ,  0.06762423,  0.01233003, -0.09266902,  0.03491845,
         0.45023501]]), array([ 1087.93076808,  1087.93076938,  1087.93077914,  1087.93078332,
        1087.93078356,  1087.93078357,  1087.93080231]))
           fun: 1087.9307680808349
       message: 'Optimization terminated successfully.'
          nfev: 1109
           nit: 693
        status: 0
       success: True
             x: a

In [11]:
#OLS ESTIMATIONS

reg2 = sm.OLS(endog=df1980['ln_wage_hours'], exog=df1980[['const', 'hyrsed', 'age', 'Black', 'Others']], missing='drop')
type(reg2)
results = reg2.fit()
type(results)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          ln_wage_hours   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     86.59
Date:                Tue, 26 Sep 2017   Prob (F-statistic):           3.56e-67
Time:                        09:43:59   Log-Likelihood:                -1087.8
No. Observations:                1751   AIC:                             2186.
Df Residuals:                    1746   BIC:                             2213.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.6157      0.078     20.682      0.0

# year 1990

In [15]:
#MAXIMUM LIKELIHOOD ESTIMATIONS 

def regressLL(params):
    #the initial parameter guesses
    b0 = params[0]
    b1 = params[1]
    b2 = params[2]
    b3 = params[3]
    b4 = params[4]
    sd = params[5]

    # Calculate the predicted values from the initial parameter guesses
    yPred = b0 + b1 * df1990['hyrsed'] + b2 * df1990['age'] + b3 * df1990['Black'] + b4 * df1990['Others'] 

    # Calculate the negative log-likelihood as the negative sum of the log of a normal
    # PDF where the observed values are normally distributed around the mean (yPred)
    # with a standard deviation of sd
    logLik = -np.sum( stats.norm.logpdf(df1990['ln_wage_hours'], loc=yPred, scale=sd) )

    # Tell the function to return the NLL (this is what will be minimized)
    return(logLik)

# Make a list of initial parameter guesses (for the betas and  sd)    
initParams = [1, -6, 0, 0.5, -1, 1]


results = opt.minimize(regressLL, initParams, method='Nelder-Mead')

print(results)

 final_simplex: (array([[ 1.30462639,  0.08638606,  0.01316871, -0.24457103, -0.3620414 ,
         0.48335431],
       [ 1.30468403,  0.08638236,  0.01316838, -0.24452079, -0.36202235,
         0.4833606 ],
       [ 1.30467334,  0.08638255,  0.01316828, -0.24450477, -0.36204569,
         0.48334895],
       [ 1.30458969,  0.08638876,  0.01316874, -0.24458648, -0.36206739,
         0.48335481],
       [ 1.30454699,  0.08639148,  0.0131684 , -0.24456418, -0.36213051,
         0.48335514],
       [ 1.30464142,  0.08638492,  0.01316874, -0.24456794, -0.36202804,
         0.48335338],
       [ 1.3046473 ,  0.08638611,  0.01316813, -0.24449732, -0.36207938,
         0.48334934]]), array([ 1350.8613775 ,  1350.86141106,  1350.86141926,  1350.86142371,
        1350.86143442,  1350.86143753,  1350.8614506 ]))
           fun: 1350.8613774976918
       message: 'Optimization terminated successfully.'
          nfev: 791
           nit: 475
        status: 0
       success: True
             x: ar

In [16]:
#OLS ESTIMATIONS

reg3 = sm.OLS(endog=df1990['ln_wage_hours'], exog=df1990[['const', 'hyrsed', 'age', 'Black', 'Others']], missing='drop')
type(reg3)
results = reg3.fit()
type(results)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          ln_wage_hours   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.221
Method:                 Least Squares   F-statistic:                     139.8
Date:                Tue, 26 Sep 2017   Prob (F-statistic):          3.87e-105
Time:                        09:44:24   Log-Likelihood:                -1340.6
No. Observations:                1953   AIC:                             2691.
Df Residuals:                    1948   BIC:                             2719.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0659      0.086     12.328      0.0

# year 2000

In [17]:
#MAXIMUM LIKELIHOOD ESTIMATIONS 

#year 2000
def regressLL(params):
    #the initial parameter guesses
    b0 = params[0]
    b1 = params[1]
    b2 = params[2]
    b3 = params[3]
    b4 = params[4]
    sd = params[5]

    # Calculate the predicted values from the initial parameter guesses
    yPred = b0 + b1 * df2000['hyrsed'] + b2 * df2000['age'] + b3 * df2000['Black'] + b4 * df2000['Others'] 

    # Calculate the negative log-likelihood as the negative sum of the log of a normal
    # PDF where the observed values are normally distributed around the mean (yPred)
    # with a standard deviation of sd
    logLik = -np.sum( stats.norm.logpdf(df2000['ln_wage_hours'], loc=yPred, scale=sd) )

    # Tell the function to return the NLL (this is what will be minimized)
    return(logLik)

# Make a list of initial parameter guesses (for the betas and  sd)    
initParams = [1, -6, 0, 0.5, -1, 1]


results = opt.minimize(regressLL, initParams, method='Nelder-Mead')


print(results)

 final_simplex: (array([[ 1.08651627,  0.10184637,  0.0154328 , -0.2248104 , -0.13485393,
         0.54426555],
       [ 1.08371876,  0.1019846 ,  0.01545897, -0.22606278, -0.13323235,
         0.54428586],
       [ 1.08031869,  0.10218554,  0.01546847, -0.22786631, -0.13530487,
         0.54390869],
       [ 1.08660121,  0.1017749 ,  0.01545196, -0.2306614 , -0.13568014,
         0.54389321],
       [ 1.08494782,  0.1019401 ,  0.01544732, -0.22965406, -0.1361226 ,
         0.54455079],
       [ 1.08639284,  0.10178137,  0.01546651, -0.22768458, -0.13184806,
         0.54384963],
       [ 1.08288171,  0.10211296,  0.01545125, -0.22619264, -0.13544317,
         0.54388861]]), array([ 2021.53351248,  2021.53364389,  2021.53431013,  2021.53469512,
        2021.53504328,  2021.53514092,  2021.53774385]))
           fun: 2021.5335124838991
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 1200
           nit: 775
        status: 1
       success: Fa

In [18]:
reg4 = sm.OLS(endog=df2000['ln_wage_hours'], exog=df2000[['const', 'hyrsed', 'age', 'Black', 'Others']], missing='drop')
type(reg4)
results = reg4.fit()
type(results)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          ln_wage_hours   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     159.2
Date:                Tue, 26 Sep 2017   Prob (F-statistic):          1.69e-121
Time:                        09:44:36   Log-Likelihood:                -2012.4
No. Observations:                2503   AIC:                             4035.
Df Residuals:                    2498   BIC:                             4064.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1638      0.084     13.926      0.0

# whole data set

MLE estimation for whole data set

In [19]:
def regressLL(params):
    #the initial parameter guesses
    b0 = params[0]
    b1 = params[1]
    b2 = params[2]
    b3 = params[3]
    b4 = params[4]
    sd = params[5]

    # Calculate the predicted values from the initial parameter guesses
    yPred = b0 + b1 * data_final['hyrsed'] + b2 * data_final['age'] + b3 * data_final['Black'] + b4 * data_final['Others'] 

# Calculate the negative log-likelihood as the negative sum of the log of a normal PDF where the observed values are normally distributed around the mean (yPred) with a standard deviation of sd
    logLik = -np.sum(stats.norm.logpdf(data_final['ln_wage_hours'], loc=yPred, scale=sd) )
    
# Tell the function to return the NLL (this is what will be minimized)
    return(logLik)

# Make a list of initial parameter guesses (for the betas and  sd)    
initParams = [1, -6, 0, 0.5, -2, 0.9]



results = opt.minimize(regressLL, initParams, method='Nelder-Mead')

print(results)

 final_simplex: (array([[  1.37534287e+00,   7.88017428e-02,   1.48660246e-02,
         -1.60544497e-01,   1.41802468e-03,   4.90805784e-01],
       [  1.37531014e+00,   7.88041812e-02,   1.48657827e-02,
         -1.60510856e-01,   1.36506658e-03,   4.90800317e-01],
       [  1.37529843e+00,   7.88032261e-02,   1.48661847e-02,
         -1.60541567e-01,   1.43981319e-03,   4.90800387e-01],
       [  1.37530374e+00,   7.88039124e-02,   1.48658619e-02,
         -1.60484952e-01,   1.44356689e-03,   4.90796574e-01],
       [  1.37536619e+00,   7.88001450e-02,   1.48658452e-02,
         -1.60469610e-01,   1.51406082e-03,   4.90794237e-01],
       [  1.37527996e+00,   7.88064986e-02,   1.48660425e-02,
         -1.60502811e-01,   1.45414407e-03,   4.90796960e-01],
       [  1.37533271e+00,   7.88024371e-02,   1.48661618e-02,
         -1.60546720e-01,   1.41415597e-03,   4.90789896e-01]]), array([ 38516.32529258,  38516.32530182,  38516.32530404,  38516.32530433,
        38516.32532534,  38516.

OLS estimation for whole data set

In [20]:
reg0 = sm.OLS(endog=data_final['ln_wage_hours'], exog=data_final[['const', 'hyrsed', 'age', 'Black', 'Others']], missing='drop')
type(reg0)
results = reg0.fit()
type(results)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          ln_wage_hours   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     3108.
Date:                Tue, 26 Sep 2017   Prob (F-statistic):               0.00
Time:                        09:48:16   Log-Likelihood:                -38513.
No. Observations:               54491   AIC:                         7.704e+04
Df Residuals:                   54486   BIC:                         7.708e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3982      0.015     90.368      0.0

# Interpret the coeﬃcient β1. How do the returns to education change over time in these data?

Returns to education is increasing over time.These results confirm the importance of investing in human capital for individuals.


__OLS interpretation is the following:__
__In Year 1970__: “if we change education by 1 (unit), we’d expect our log_wages to change by 6.67 percent”.
__In Year 1980__: “if we change education by 1 (unit), we’d expect our log_wages to change by 6.75 percent”.
__In Year 1990__: “if we change education by 1 (unit), we’d expect our log_wages to change by 9.93 percent”.
__In Year 2000__: “if we change education by 1 (unit), we’d expect our log_wages to change by 10.99 percent”.

__Estimations with Nelder-Mead method is the following:__
__In Year 1970__: “if we change education by 1 (unit), we’d expect our log_wages to change by 6.64 percent”.
__In Year 1980__: “if we change education by 1 (unit), we’d expect our log_wages to change by 6.7 percent”.
__In Year 1990__: “if we change education by 1 (unit), we’d expect our log_wages to change by 8.64 percent”.
__In Year 2000__: “if we change education by 1 (unit), we’d expect our log_wages to change by 10.1 percent”.

# EXTRA: UNSUCCESFUL SLSQP METHOD ATTEMPT
year 2000

In [24]:


#year 2000
def regressLL(params):
    #the initial parameter guesses
    b0 = params[0]
    b1 = params[1]
    b2 = params[2]
    b3 = params[3]
    b4 = params[4]
    sd = params[5]

    yPred = b0 + b1 * df2000['hyrsed'] + b2 * df2000['age'] + b3 * df2000['Black'] + b4 * df2000['Others'] 
    logLik = -np.sum( stats.norm.logpdf(df2000['ln_wage_hours'], loc=yPred, scale=sd) )

    # Tell the function to return the NLL (this is what will be minimized)
    return(logLik)

b8=[3,0.1,0.2,0.05,0.001,0.05]
bnds = ((-np.inf, np.inf),(-np.inf, np.inf), (-np.inf, np.inf),(-np.inf, np.inf), (-np.inf, np.inf), (0, np.inf))
bnds = ((0, None), (0, None), (0, None), (None, None), (None, None), (None, None))
results1 = opt.minimize(regressLL,b8,bounds=bnds, method='SLSQP')
print(results1)

     fun: 40022.10131551729
     jac: array([ 0.,  0.,  0.,  0.,  0.,  0.])
 message: 'Optimization terminated successfully.'
    nfev: 146
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([  3.00000000e+00,  -2.32051575e+04,  -4.22631054e+04,
        -4.78431759e+01,  -2.67007643e+01,   1.94897853e+06])
